In [1]:
from ragcar import Ragcar
from ragcar.utils import PromptTemplate

In [2]:
Ragcar.available_models("text_generation")

'Available models for text_generation are ([src]: openai, [model]: gpt-4-turbo-preview, gpt-4, gpt-3.5-turbo, MODELS_SUPPORTED(https://platform.openai.com/docs/models)), ([src]: clova, [model]: YOUR_MODEL(https://www.ncloud.com/product/aiService/clovaStudio))'

# Parameters for Prediction
* prompt_template: PromptTemplate,
* max_tokens: int=1000,
* temperature: float=0.1 if src == 'clova' else 0.,
* top_p: float=1.,
* frequency_penalty: float=0.1 if src == 'clova' else 0., 
* presence_penalty: int=0 if src == 'clova' else float=0.,
* stop_before: Optional[list] = None,
* functions: Optional[list] = None,
* stream: Optional[bool] = False, 
* use_async: bool = False, 
* formatting: Optional[bool] = False

# Models

## [OpenAI](https://platform.openai.com/docs/api-reference)
`.env` 파일 또는 환경 변수로 `OPENAI_API_KEY`를 설정하거나 다음과 같이 직접 변수를 입력합니다.
* model_n: 모델명
* api_key: OPENAI_API_KEY

In [3]:
Ragcar.available_model_fields("openai")

'Available fields for openai are ([field]: model_n, [type]: str), ([field]: api_key, [type]: str)'

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
generator = Ragcar(
    tool="text_generation", 
    src="openai",
    model={
        "model_n": "gpt-4-turbo-preview",
        "api_key": os.getenv('OPENAI_API_KEY')
    },
    prompt_template=PromptTemplate("사용자: {input} 수도는?\nAI:"), 
    max_tokens=100,
)

In [6]:
generator(input="대한민국")

<OpenAIObject chat.completion id=chatcmpl-92adM64BK1xS76vUFVn9AxCDHQ7BN at 0x7f9b488f0770> JSON: {
  "id": "chatcmpl-92adM64BK1xS76vUFVn9AxCDHQ7BN",
  "object": "chat.completion",
  "created": 1710404504,
  "model": "gpt-4-0125-preview",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\ub300\ud55c\ubbfc\uad6d\uc758 \uc218\ub3c4\ub294 \uc11c\uc6b8\uc785\ub2c8\ub2e4."
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 23,
    "completion_tokens": 15,
    "total_tokens": 38
  },
  "system_fingerprint": "fp_31c0f205d1"
}

### Formatting

In [7]:
generator = Ragcar(
    tool="text_generation", 
    src="openai",
    prompt_template=PromptTemplate("사용자: {input} 수도는?\nAI:"), 
    max_tokens=100,
    formatting=True
)

In [8]:
generator(input="대한민국")

{'id': 'openai-0548e3c2-dbf9-4f37-95c9-13f3653673c0',
 'model': 'gpt-4-turbo-preview',
 'content': '대한민국의 수도는 서울입니다.',
 'finish_reason': 'stop',
 'input_tokens': 23,
 'output_tokens': 15,
 'total_tokens': 38,
 'predicted_cost': 0.0015899999999999998,
 'response_time': 5.023842096328735}

### Sync VS Async

In [9]:
generator = Ragcar(
    tool="text_generation", 
    src="openai",
    model="gpt-3.5-turbo",
    prompt_template=PromptTemplate("사용자: {input} 수도는?\nAI:"), 
    max_tokens=100,
    use_async=False
)

async_generator = Ragcar(
    tool="text_generation", 
    src="openai",
    model="gpt-3.5-turbo",
    prompt_template=PromptTemplate("사용자: {input} 수도는?\nAI:"), 
    max_tokens=100,
    use_async=True
)

In [10]:
import asyncio
import time

countries = ["대한민국", "중국", "일본"]

print("\n-------------------------\n")
print("Sync")
start = time.time()
result = [generator(input=c) for c in countries]
end = time.time() - start
print([i.choices[0].message.content for i in result])
print(f"Total time: {end}")

print("\n-------------------------\n")
print("Async")
start = time.time()
tasks = [async_generator(input=c) for c in countries]
result = await asyncio.gather(*tasks)
end = time.time() - start
print([i.choices[0].message.content for i in result])
print(f"Total time: {end}")


-------------------------

Sync
['서울입니다.', '베이징입니다.', '도쿄입니다.']
Total time: 2.4002315998077393

-------------------------

Async
['서울입니다.', '베이징입니다.', '도쿄입니다.']
Total time: 1.0498955249786377


### Streaming

In [11]:
generator = Ragcar(
    tool="text_generation", 
    src="openai",
    model="gpt-3.5-turbo",
    prompt_template=PromptTemplate("사용자: {input} 수도는?\nAI:"), 
    max_tokens=100,
    use_async=False,
    stream=True
)

In [12]:
res = generator(input="대한민국")

for chunk in res:
    print(chunk)

{
  "id": "openai-e3a60b6b-cd84-4ddf-8c90-42f166221bbc",
  "object": "chat.completion.chunk",
  "created": 1710404515,
  "model": "gpt-3.5-turbo-0125",
  "system_fingerprint": "fp_4f0b692a78",
  "choices": [
    {
      "index": 0,
      "delta": {
        "role": "assistant",
        "content": ""
      },
      "logprobs": null,
      "finish_reason": null
    }
  ]
}
{
  "id": "openai-e3a60b6b-cd84-4ddf-8c90-42f166221bbc",
  "object": "chat.completion.chunk",
  "created": 1710404515,
  "model": "gpt-3.5-turbo-0125",
  "system_fingerprint": "fp_4f0b692a78",
  "choices": [
    {
      "index": 0,
      "delta": {
        "content": "\uc11c"
      },
      "logprobs": null,
      "finish_reason": null
    }
  ]
}
{
  "id": "openai-e3a60b6b-cd84-4ddf-8c90-42f166221bbc",
  "object": "chat.completion.chunk",
  "created": 1710404515,
  "model": "gpt-3.5-turbo-0125",
  "system_fingerprint": "fp_4f0b692a78",
  "choices": [
    {
      "index": 0,
      "delta": {
        "content": "\uc6b8"


##### With formatting

In [13]:
generator = Ragcar(
    tool="text_generation", 
    src="openai",
    model="gpt-3.5-turbo",
    prompt_template=PromptTemplate("사용자: {input} 수도는?\nAI:"), 
    max_tokens=100,
    use_async=False,
    stream=True,
    formatting=True
)

In [14]:
res = generator(input="대한민국")

for chunk in res:
    print(chunk)

{'id': 'openai-fe922e2d-1ad1-4ef5-9c24-3d115168499d', 'event': 'chunk', 'content': '서'}
{'id': 'openai-fe922e2d-1ad1-4ef5-9c24-3d115168499d', 'event': 'chunk', 'content': '울'}
{'id': 'openai-fe922e2d-1ad1-4ef5-9c24-3d115168499d', 'event': 'chunk', 'content': '입니다'}
{'id': 'openai-fe922e2d-1ad1-4ef5-9c24-3d115168499d', 'event': 'chunk', 'content': '.'}
{'id': 'openai-fe922e2d-1ad1-4ef5-9c24-3d115168499d', 'event': 'finish', 'data': {'id': 'openai-fe922e2d-1ad1-4ef5-9c24-3d115168499d', 'model': 'gpt-3.5-turbo', 'content': '서울입니다.', 'finish_reason': 'stop', 'input_tokens': 21, 'output_tokens': 5, 'total_tokens': 26, 'predicted_cost': 3.0999999999999995e-05, 'response_time': 0.6366674900054932}}


##### With use_async

In [15]:
generator = Ragcar(
    tool="text_generation", 
    src="openai",
    model="gpt-3.5-turbo",
    prompt_template=PromptTemplate("사용자: {input} 수도는?\nAI:"), 
    max_tokens=100,
    use_async=True,
    stream=True,
    formatting=True
)

In [16]:
res = await generator(input="대한민국")

async for chunk in res:
    print(chunk)

{'id': 'openai-33210bc7-889e-43b2-937b-6416c995d280', 'event': 'chunk', 'content': ''}
{'id': 'openai-33210bc7-889e-43b2-937b-6416c995d280', 'event': 'chunk', 'content': '서'}
{'id': 'openai-33210bc7-889e-43b2-937b-6416c995d280', 'event': 'chunk', 'content': '울'}
{'id': 'openai-33210bc7-889e-43b2-937b-6416c995d280', 'event': 'chunk', 'content': '입니다'}
{'id': 'openai-33210bc7-889e-43b2-937b-6416c995d280', 'event': 'chunk', 'content': '.'}
{'id': 'openai-33210bc7-889e-43b2-937b-6416c995d280', 'event': 'finish', 'data': {'id': 'openai-33210bc7-889e-43b2-937b-6416c995d280', 'model': 'gpt-3.5-turbo', 'content': '서울입니다.', 'finish_reason': 'stop', 'input_tokens': 21, 'output_tokens': 5, 'total_tokens': 26, 'predicted_cost': 3.0999999999999995e-05, 'response_time': 0.6105508804321289}}


### Chat Prompt template

In [17]:
import logging

logging.basicConfig(level=logging.INFO)

In [18]:
Prompt_template = PromptTemplate(
    user_prefix="사용자", 
    assistant_prefix="AI"
)
Prompt_template.system("사용자의 질문에 대한 정답만 출력합니다.")
Prompt_template.user("미국 수도는?")
Prompt_template.assistant("워싱턴")
Prompt_template.user("{input} 수도는?")
print(f"Prompt template:\n{Prompt_template.get_prompt()}")

generator = Ragcar(
    tool="text_generation", 
    src="openai",
    prompt_template=Prompt_template, 
    max_tokens=100,
    formatting=True
)

Prompt template:
[{'role': 'system', 'content': '사용자의 질문에 대한 정답만 출력합니다.'}, {'role': 'user', 'content': '사용자: 미국 수도는?'}, {'role': 'assistant', 'content': 'AI: 워싱턴'}, {'role': 'user', 'content': '사용자: {input} 수도는?'}]


In [19]:
generator(input="대한민국")

INFO:ragcar.models.base:{
    "id": "openai-92e24264-aea2-4be3-9502-8a2e2d45dfaf",
    "model": "gpt-4-turbo-preview",
    "parameters": {
        "messages": [
            {
                "role": "system",
                "content": "사용자의 질문에 대한 정답만 출력합니다."
            },
            {
                "role": "user",
                "content": "사용자: 미국 수도는?"
            },
            {
                "role": "assistant",
                "content": "AI: 워싱턴"
            },
            {
                "role": "user",
                "content": "사용자: 대한민국 수도는?\nAI: "
            }
        ],
        "max_tokens": 100,
        "temperature": 0.0,
        "top_p": 1.0,
        "frequency_penalty": 0.0,
        "presence_penalty": 2.0,
        "stream": false
    }
}
INFO:ragcar.models.base:{
    "id": "openai-92e24264-aea2-4be3-9502-8a2e2d45dfaf",
    "message": "Request completed successfully"
}
INFO:ragcar.models.openai:{
    "id": "chatcmpl-92adatYN8rfVskTLaj9aLbHpkvqbF",
    "obj

{'id': 'openai-92e24264-aea2-4be3-9502-8a2e2d45dfaf',
 'model': 'gpt-4-turbo-preview',
 'content': '서울',
 'finish_reason': 'stop',
 'input_tokens': 73,
 'output_tokens': 3,
 'total_tokens': 76,
 'predicted_cost': 0.0023699999999999997,
 'response_time': 0.6759653091430664}

### Function calling

In [20]:
functions=[
    {
        "name": "retriever",
        "description": "Retrieve relevant data based on a query",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The query to retrieve relevant data"
                }
            },
            "required": ["query"]
        }
    }
]

generator = Ragcar(
    tool="text_generation", 
    src="openai",
    prompt_template=PromptTemplate("사용자: {input} 수도는?\nAI:"), 
    max_tokens=100,
    functions=functions,
    formatting=True
)

In [21]:
action = generator(input="대한민국")
print(action)
print(action['content'].function_call.arguments)

INFO:ragcar.models.base:{
    "id": "openai-d3d0f2e1-bcac-498b-9b55-5c3421268efe",
    "model": "gpt-4-turbo-preview",
    "parameters": {
        "messages": [
            {
                "role": "user",
                "content": "사용자: 대한민국 수도는?\nAI:"
            }
        ],
        "max_tokens": 100,
        "temperature": 0.0,
        "top_p": 1.0,
        "frequency_penalty": 0.0,
        "presence_penalty": 2.0,
        "functions": [
            {
                "name": "retriever",
                "description": "Retrieve relevant data based on a query",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The query to retrieve relevant data"
                        }
                    },
                    "required": [
                        "query"
                    ]
                }
    

{'id': 'openai-d3d0f2e1-bcac-498b-9b55-5c3421268efe', 'model': 'gpt-4-turbo-preview', 'content': <OpenAIObject at 0x7f9b487b2c50> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "retriever",
    "arguments": "{\"query\":\"\ub300\ud55c\ubbfc\uad6d \uc218\ub3c4\"}"
  }
}, 'finish_reason': 'function_call', 'input_tokens': 71, 'output_tokens': 22, 'total_tokens': 93, 'predicted_cost': 0.00345, 'response_time': 3.1882152557373047}
{"query":"대한민국 수도"}


### gpt-3

In [22]:
generator = Ragcar(
    tool="text_generation", 
    src="openai",
    model="davinci-002",
    prompt_template=PromptTemplate("User: What is the capital of {input}?\nAI:"), 
    max_tokens=10,
    stop_before=['User:', 'AI:'],
    formatting=True
)

In [23]:
generator(input="republic of korea")

INFO:ragcar.models.base:{
    "id": "openai-af8e6f46-4409-48b8-ba4d-f1e1e3e3d79e",
    "model": "davinci-002",
    "parameters": {
        "prompt": "User: What is the capital of republic of korea?\nAI:",
        "max_tokens": 10,
        "temperature": 0.0,
        "top_p": 1.0,
        "frequency_penalty": 0.0,
        "presence_penalty": 2.0,
        "stop": [
            "User:",
            "AI:"
        ],
        "stream": false
    }
}
INFO:ragcar.models.base:{
    "id": "openai-af8e6f46-4409-48b8-ba4d-f1e1e3e3d79e",
    "message": "Request completed successfully"
}
INFO:ragcar.models.openai:{
    "id": "cmpl-92adfNOD6WZFXEcDpwwsGF1oVr9nK",
    "object": "text_completion",
    "created": 1710404523,
    "model": "davinci-002",
    "choices": [
        {
            "text": " Seoul\n",
            "index": 0,
            "logprobs": null,
            "finish_reason": "stop"
        }
    ],
    "usage": {
        "prompt_tokens": 14,
        "completion_tokens": 2,
        "tota

{'id': 'openai-af8e6f46-4409-48b8-ba4d-f1e1e3e3d79e',
 'model': 'davinci-002',
 'content': 'Seoul',
 'finish_reason': 'stop',
 'input_tokens': 14,
 'output_tokens': 2,
 'total_tokens': 16,
 'predicted_cost': 3.2e-05,
 'response_time': 0.3503868579864502}

## HyperCLOVA X 모델
`.env` 파일 또는 환경 변수로 `X-NCP-APIGW-API-KEY`, `X-NCP-CLOVASTUDIO-API-KEY`를 설정하거나 다음과 같이 직접 변수를 입력합니다. 
HyperCLOVA API 사용방법은 [여기서](https://guide.ncloud-docs.com/docs/clovastudio-playground01) 참고해주세요.
* model_n: API URL
* api_key: X-NCP-APIGW-API-KEY
* app_key: X-NCP-CLOVASTUDIO-API-KEY

**⚠️ 파라미터 명 변경:**
* top_k 대신 presence_penalty를 사용해주세요.
* repeat_penalty 대신 frequency_penalty를 사용해주세요.


**⚠️ 파라미터 값 범위:**
* 0.0 < temperature < 1.0
* 0.0 < top_p < 1.0
* 0 < presence_penalty < 128
* 0.0 < frequency_penalty < 10.0

In [24]:
Ragcar.available_model_fields('clova')

'Available fields for clova are ([field]: model_n, [type]: str), ([field]: api_key, [type]: str), ([field]: app_key, [type]: str)'

In [25]:
generator = Ragcar(
    tool="text_generation", 
    src="clova",
    model={
        "model_n": "https://clovastudio.stream.ntruss.com/testapp/v1/chat-completions/HCX-002",
        "api_key": os.getenv("X-NCP-APIGW-API-KEY"),
        "app_key": os.getenv("X-NCP-CLOVASTUDIO-API-KEY")
    },
    prompt_template=PromptTemplate("사용자: {input} 수도는?\nAI:"), 
    max_tokens=100,
    formatting=True
)

In [26]:
generator(input="대한민국")

INFO:ragcar.models.base:{
    "id": "clova-53147d01-2af4-4d41-afab-79ad9f23a11f",
    "model": "https://clovastudio.stream.ntruss.com/testapp/v1/chat-completions/HCX-002",
    "parameters": {
        "messages": [
            {
                "role": "user",
                "content": "사용자: 대한민국 수도는?\nAI:"
            }
        ],
        "maxTokens": 100,
        "temperature": 0.1,
        "topP": 1.0,
        "topK": 2,
        "repeatPenalty": 0.1,
        "stopBefore": []
    }
}
INFO:ragcar.models.base:{
    "id": "clova-53147d01-2af4-4d41-afab-79ad9f23a11f",
    "message": "Request completed successfully"
}
INFO:ragcar.models.clova:{
    "status": {
        "code": "20000",
        "message": "OK"
    },
    "result": {
        "message": {
            "role": "assistant",
            "content": "서울"
        },
        "inputLength": 12,
        "outputLength": 3,
        "stopReason": "stop_before",
        "seed": 1867422411,
        "aiFilter": []
    }
}


{'id': 'clova-53147d01-2af4-4d41-afab-79ad9f23a11f',
 'model': 'HCX-002',
 'content': '서울',
 'finish_reason': 'stop_before',
 'input_tokens': 12,
 'output_tokens': 3,
 'total_tokens': 15,
 'predicted_cost': 0.44999999999999996,
 'response_time': 0.4274263381958008,
 'ai_filter': []}

In [27]:
generator = Ragcar(
    tool="text_generation", 
    src="clova",
    model="https://clovastudio.stream.ntruss.com/testapp/v1/chat-completions/HCX-002",
    prompt_template=PromptTemplate("사용자: {input} 수도는?\nAI:"), 
    max_tokens=100,
    stream=True,
    formatting=True
)

In [28]:
res = generator(input="대한민국")

for chunk in res:
    print(chunk)

INFO:ragcar.models.base:{
    "id": "clova-d8171a24-e5b7-4eec-a5da-acd8022c0593",
    "model": "https://clovastudio.stream.ntruss.com/testapp/v1/chat-completions/HCX-002",
    "parameters": {
        "messages": [
            {
                "role": "user",
                "content": "사용자: 대한민국 수도는?\nAI:"
            }
        ],
        "maxTokens": 100,
        "temperature": 0.1,
        "topP": 1.0,
        "topK": 2,
        "repeatPenalty": 0.1,
        "stopBefore": []
    }
}
INFO:ragcar.models.base:{
    "id": "clova-d8171a24-e5b7-4eec-a5da-acd8022c0593",
    "message": "Request completed successfully"
}


## AI 챗봇

In [29]:
logging.getLogger().setLevel(logging.ERROR)

In [30]:
class SimpleChatbot:
    def __init__(self, generator, conversation_template):
        self.generator = generator
        self.conversation_template = conversation_template

    def ask(self, question):
        print(self.conversation_template.format_text())
        
        # Generate the response using the Toolva generator
        answer = self.generator(input=question, conversation=self.conversation_template.format_text())
        
        self.conversation_template.user(question)
        self.conversation_template.assistant(answer['content'])
        
        return answer

In [31]:
prompt_template = PromptTemplate(user_prefix="Q", assistant_prefix="A")
prompt_template.system((
    "너는 AI 여행 플래너야. 사용자의 질문에 대한 적절한 답변을 생성해\n\n"
    "conversation:\n"
    "{conversation}"
))
prompt_template.user("{input}")

generator = Ragcar(
    tool="text_generation", 
    src="openai",
    model="gpt-3.5-turbo",
    prompt_template=prompt_template, 
    max_tokens=300,
    formatting=True
)

conversation_template = PromptTemplate(user_prefix="Q", assistant_prefix="A")

chatbot = SimpleChatbot(generator, conversation_template)

In [32]:
chatbot.ask("요즘 여행가기 어디가 좋을까?")

{'id': 'openai-96b5fe37-ebc2-4c49-b07e-98143fff74a2',
 'model': 'gpt-3.5-turbo',
 'content': '요즘 여행하기 좋은 곳은 자연 속으로 힐링을 즐길 수 있는 여행지입니다. 예를 들어 국내에서는 강원도의 평창이나 제주도, 해외로는 뉴질랜드의 풍부한 자연 경치가 매력적인 여행지일 수 있습니다. 원하는 여행 스타일과 관심사에 맞게 선택하시면 좋을 것 같아요.',
 'finish_reason': 'stop',
 'input_tokens': 73,
 'output_tokens': 135,
 'total_tokens': 208,
 'predicted_cost': 0.000343,
 'response_time': 3.0994982719421387}

In [33]:
chatbot.ask("여름에 갈거고 나는 먹는걸 좋아해")

Q: 요즘 여행가기 어디가 좋을까?
A: 요즘 여행하기 좋은 곳은 자연 속으로 힐링을 즐길 수 있는 여행지입니다. 예를 들어 국내에서는 강원도의 평창이나 제주도, 해외로는 뉴질랜드의 풍부한 자연 경치가 매력적인 여행지일 수 있습니다. 원하는 여행 스타일과 관심사에 맞게 선택하시면 좋을 것 같아요.


{'id': 'openai-b2a1b959-5d72-45f7-bcb0-6920af2265fb',
 'model': 'gpt-3.5-turbo',
 'content': '여름에 먹는 것을 즐기며 여행하기 좋은 곳으로는 해안가나 섬이 있는 지역이 좋을 수 있어요. 국내에서는 부산의 맛집 투어나 제주도의 해산물 요리를 즐길 수 있고, 해외로는 태국의 방콕이나 이탈리아의 시칠리아 섬 등이 추천됩니다. 현지 음식을 맛보면서 여행을 즐기는 건 어떨까요?',
 'finish_reason': 'stop',
 'input_tokens': 232,
 'output_tokens': 155,
 'total_tokens': 387,
 'predicted_cost': 0.000542,
 'response_time': 3.5681326389312744}

### RAG(Retrieval-Augmented Generation)

#### Data
* 한국관광공사_무장애 여행 정보_GW(https://www.data.go.kr/data/15101897/openapi.do)

In [34]:
import pandas as pd

data = pd.read_csv("corpus_sample.csv")
data

,contentid,title,corpus
0,1433504,가경 터미널시장,"가경 터미널시장, 충북 청주시 흥덕구 가경동에 위치한 상설 재래시장이다. 논밭이었던..."
1,126273,가계해변,"가계해변, 바닷물이 갈라지는 현대판 모세의 기적으로 유명한 회동국민관광지 내에 있는..."
2,129854,가나아트센터,"가나아트센터, 1983년에 개관한 가나아트갤러리는 국내외 3백여 회의 기획전시 및 ..."
3,129194,가나아트파크,"가나아트파크, 가나아트파크(전, 장흥아트파크)는 1984년 국내 최초 사립미술관인 ..."
4,2777865,가남체육공원,"가남체육공원, 가남체육공원은 가남면 태평리에 위치한 주민들을 위한 다목적 체육활동 ..."
...,...,...,...
5280,1839801,[제주올레 14코스] 저지-한림 올레,"[제주올레 14코스] 저지-한림 올레, 고요하고 아늑한 초록의 숲길 올레와 시원하게..."
5281,1839536,[제주올레 4코스] 표선-남원 올레,"[제주올레 4코스] 표선-남원 올레, 눈부신 백사장에서 시작되는 아름다운 해안 올레..."
5282,1839552,[제주올레 5코스] 남원-쇠소깍 올레,"[제주올레 5코스] 남원-쇠소깍 올레, 오감을 활짝 열고 걷는 바당올레와 마을 올레..."
5283,1839621,[제주올레 6코스] 쇠소깍-제주올레 여행자센터 올레,"[제주올레 6코스] 쇠소깍-제주올레 여행자센터 올레, 쇠소깍 다리를 출발하여 서귀포..."


In [35]:
encoder = Ragcar(tool="sentence_embedding", src="model_name_or_path", model="leewaay/klue-roberta-large-klueNLI-klueSTS-MSL512")
embeddings = encoder(data["corpus"].tolist(), batch_size=100, convert_to_tensor=True)

In [36]:
class RagChatbotAgent:
    def __init__(
        self, 
        candidates, 
        embeddings, 
        retriever, 
        conversation_template
    ):
        self.candidates = candidates
        self.embeddings = embeddings
        self.retriever = retriever
        self.conversation_template = conversation_template

    def ask(self, question, generator):
        hits = self.retriever(question, self.embeddings, top_k=5, m1in_score=0.3)[0]
        data = '\n'.join([str(self.candidates.iloc[hit['corpus_id']].to_dict()) for hit in hits])
        
        # Generate the response using the Toolva generator
        answer = generator(input=question, data=data, conversation=self.conversation_template.format_text())
        
        return answer

In [37]:
retriever = Ragcar(tool="semantic_search", src="model_name_or_path", model="leewaay/klue-roberta-large-klueNLI-klueSTS-MSL512")
conversation_template = PromptTemplate(user_prefix="Q", assistant_prefix="A")

chatbot = RagChatbotAgent(data, embeddings, retriever, conversation_template)

#### OpenAI VS HyperCLOVA X

In [40]:
# Prompt
prompt_template = PromptTemplate(user_prefix="Q", assistant_prefix="A")
prompt_template.system((
    "너는 AI 여행 플래너야. data를 기반으로 사용자의 질문에 대한 적절한 답변을 생성해\n\n"
    "data:\n"
    "{data}\n\n"
    "conversation:\n"
    "{conversation}"
))
prompt_template.user((
    "{input}"
))

# Models
openai = Ragcar(
    tool="text_generation", 
    src="openai",
    model="gpt-3.5-turbo",
    prompt_template=prompt_template, 
    max_tokens=1000,
    formatting=True
)

clovax = Ragcar(
    tool="text_generation", 
    src="clova",
    model="https://clovastudio.stream.ntruss.com/testapp/v1/chat-completions/HCX-003",
    prompt_template=prompt_template, 
    max_tokens=1000,
    temperature=0.1,
    top_p=0.9,
    frequency_penalty=1,
    formatting=True
)

In [41]:
question = "부산 여행지 추천해줘"

print("\n-------------------------\n")
print("OpenAI Answer")
print(chatbot.ask(question, openai))

print("\n-------------------------\n")
print("HyperCLOVA X Answer")
print(chatbot.ask(question, clovax))


-------------------------

OpenAI Answer
{'id': 'openai-876c2036-0cde-4167-9e26-876c72348459', 'model': 'gpt-3.5-turbo', 'content': '부산 여행을 계획 중이시군요! 부산에는 다양한 관광지가 있지만, 몇 가지 추천해 드릴게요. 해운대해수욕장은 부산을 대표하는 명소로 유명하며, 넓고 아름다운 백사장과 세련된 분위기의 해안선이 인상적입니다. 또한, 해운대 달맞이길은 부산에서 가장 유명한 드라이브 코스로서 푸른 바다와 동백숲이 어우러진 풍경을 감상할 수 있습니다. 마지막으로, 부산 스마트시티 전시관은 부산의 스마트시티 서비스를 체험할 수 있는 장소로 현대적인 기술을 경험해볼 수 있어요. 부산뷰호텔도 신개념의 비즈니스 모텔로 편안하고 즐거운 여행을 즐길 수 있을 거예요. 부산 여행을 즐겁게 계획해보세요!', 'finish_reason': 'stop', 'input_tokens': 2811, 'output_tokens': 327, 'total_tokens': 3138, 'predicted_cost': 0.0034649999999999998, 'response_time': 6.1814284324646}

-------------------------

HyperCLOVA X Answer
{'id': 'clova-d1a5308a-6b0c-4e82-8aef-3b53aa3b9cd2', 'model': 'HCX-003', 'content': '부산은 아름다운 자연 경관과 다양한 관광 명소가 있어 여행객들에게 인기 있는 도시입니다. 부산 여행지 추천해드리겠습니다.\n\n1.**해운대 해수욕장**: 부산의 대표적인 해수욕장으로, 백사장과 아름다운 해안선을 자랑합니다. 해수욕뿐만 아니라 다양한 축제와 행사도 열려 즐길거리가 많습니다.\n\n2.**더베이101**: 해운대에 위치한 복합문화공간으로 야경이 아름다운 곳입니다. 바다와 산 그리고 현대적인 건물들이 공존하는 공간에서 문화와